# Lecture 03 - n-Gram Language Models (advanced)

This notebook provides an interesting and to some extent funny use of Language Models: the generation of new sentences. Since a Language Model allows us to calculate the conditional probability of a word given a content (= a sequence of preceding words), we can use such a model to iteratively predict the next word given some start/seed words to generate a complete sentence.

The use case of text generation particularly highlights the differences between Language Models based on smaller or larger n-grams. 

Let's get started...

### Required Imports

Well use spaCy to handle the basics such as tokenization

In [1]:
import spacy
nlp = spacy.load("en_core_web_sm")

And we need some other things like `re` for some preprocessing/cleaning, `requests` to download our text documents that serve as corpora to train out Language Model.

In [2]:
import numpy as np
import re
import requests
from collections import defaultdict

## Creating the Language Model Class

The core component is of this notebook is of course the Language Model. Here, we warp the implementation into a Python class. This makes the implementation a bit more compact and tidy. We could also move this class into an external `.py` file and just imported into the notebook, but the class is not large, so let's keep everything into one place.

Compared to the basic notebook about Language Models, the following code needs some more complexity. However, the core requirement is still to go through a corpora and keep track of the n-gram counts that allows us the conditional probabilities of n-grams, i.e., the probability of a word given a context.

While the code below contains comments, let's address some of the basic components:

* The class variable `ngram_counter` contains the counts for each existing n-gram. This the same approach as in the basic notebook. Again, we use a `defaultdict` to make life a tad easier. As we do not have to worry about non-existing keys (i.e., n-grams) when updating the counts. The keys of the dictionaries are n-grams represented as tuples as tuples, compared to lists, are hashable and as such can be used as dictionary keys (e.g., `ngram_counter[('i', 'saw', 'that')] = 14`).

* The class variable `self.context` contains for each context the list of words/tokens that followed a context at least once. This list of words will represent our candidates when we generate new words given a context. The intuition is that we want to generate only words that we have seen at least once after the given context.

* The class method `preprocess_sentence` does some cleaning, mainly removing non-ascii characters and `\s` characters (e.g., tabs, line breaks) and replace them with a single space character. The method also uses spaCy to tokenize a sentence and return the list of tokens as result.

* The class method `get_ngrams` takes a list of tokens and generates all n-grams of the user-specified size (specified in the constructor). It also first pads the list of tokens with the start-of-sequence and end-of-sequence tokens. The amount of padding, of course, depends on the size of the ngrams.

* The class method `update` takes a sentence as input, preprocesses it and goes through all n-grams and updates the respective counts in `ngram_counter`. Note that we ignore sentence if its length is smaller then the n-gram size. In short, the method `update` is the main method we use for training our Language Model one sentence at a time.

* The class method `calc_prob` computes the conditional probability of a word/token given a context; we've already seen this in the basic notebook. Note that we don't need any smoothing as we generate new words and only pick words that we have seen at least once for a given context. We therefore do not have to deal with zero probabilities here.

* The class method `random_token` picks a random word/token given a context. As already mention, we only consider words/tokens we have seen at least once following the given context. From these candidate words/tokens, we pick randomly but not uniformly randomly. We utilize the probabilities of each candidate word/token given the index to favor words we have seen more frequently following the context.

* The class method `generate_text` does the actual generation of a sentence give an optional list of start/seed tokens. In its core, the methods generates a new word in each iteration until (a) the end-of-sentence token is generated or (b) the sentence has reached the specified maximum length. This is usually done as failsafe to avoid cases where the end-of-sentence token is never generated.

In [3]:
class NgramModel(object):

    def __init__(self, n, sos='<s>', eos='</s>'):
        self.n, self.sos, self.eos = n, sos, eos

        # We need at least bigrams for this model to work
        if self.n < 2:
            raise Exception('Size of n-grams must be at least 2!')
        
        # keeps track of how many times ngram has appeared in the text before
        self.ngram_counter = defaultdict(int)

        # Dictionary that keeps list of candidate words given context
        # When generating a text, we only pick from those candidate words
        self.context = {}

        
    def preprocess_sentence(self, s, lowercase=True):
        # Do some cleaning
        s = s.encode("ascii", "ignore").decode()
        s = re.sub(r'\s+', ' ', s)    

        # Do case folding to lowercase if specified
        if lowercase == True:
            s = s.lower()

        # Tokenize sentence and return list of tokens
        return [ t.text.strip() for t in nlp(s) if t.text.strip() != '']        

    
    def get_ngrams(self, tokens: list) -> list:
        """
        Generates all n-grams of size n given a list of tokens
        :param tokens: tokenized sentence
        :return: list of ngrams
        ngrams of tuple form: ((previous wordS!), target word)
        """
        n = self.n
        
        tokens = (n-1)*[self.sos] + tokens + (n-1)*[self.eos]
        l = [(tuple([tokens[i-p-1] for p in reversed(range(n-1))]), tokens[i]) for i in range(n-1, len(tokens))]
        return l
        
        
    def update(self, s: str) -> None:
        """
        Updates Language Model
        :param sentence: input text
        """
        tokens = self.preprocess_sentence(s)
        
        if len(tokens) < self.n:
            return
        
        ngrams = self.get_ngrams(tokens)
        for ngram in ngrams:
            self.ngram_counter[ngram] += 1.0
            prev_words, target_word = ngram
            if prev_words in self.context:
                self.context[prev_words].append(target_word)
            else:
                self.context[prev_words] = [target_word]

                
    def calc_prob(self, context, token):
        """
        Calculates probability of a token given a context
        :return: conditional probability
        """
        try:
            count_of_token = self.ngram_counter[(context, token)]
            count_of_context = float(len(self.context[context]))
            result = count_of_token / count_of_context
        except KeyError:
            result = 0.0
        return result

    
    def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        :param context:
        :return:
        """
        # Get all candidate words for the given context
        tokens_of_interest = self.context[context]
        # Get the probavilities for each ngram (context+word)
        token_probs = np.array([self.calc_prob(context, token) for token in tokens_of_interest])
        # Return a random candidate word based on the probability distribution
        # (candidate words that occur more frequently after the context have a higher prob)
        return np.random.choice(tokens_of_interest, 1, p=(token_probs / sum(token_probs)))[0]


            
    def generate_text(self, token_count: int, start_context=[]):
        """
        Iteratively generates a sentence by predicted the next word step by step
        :param token_count: number of words to be produced
        :param start_context: list of start/seed words
        :return: generated text
        """
        n = self.n
        
        # The following block merely prepares the first context; note that the context is always of size
        # (self.n - 1) so depending on the start_context (representing the start/seed words), we need to
        # pad or cut off the start_context.
        if len(start_context) == (n-1):
            context_queue = start_context.copy()
        elif len(start_context) < (n-1):
            context_queue = ((n - (len(start_context)+1)) * [self.sos]) + start_context.copy()
        elif len(start_context) > (n-1):
            context_queue = start_context[-(n-1):].copy()
        result = start_context.copy()                    
            
        # The main loop for generating words
        for _ in range(token_count):
            # Generate the next token given the current context
            obj = self.random_token(tuple(context_queue))
            # Add generated word to the result list
            result.append(obj)
            # Remove the first token from the context
            context_queue.pop(0)
            if obj == self.eos:
                # If we generate the EOS token, we can return the sentence (without the EOS token)
                return ' '.join(result[:-1])
            else:
                # Otherwise create the new context and keep generate the next word
                context_queue.append(obj)
        # Fallback if we predict more than token_count tokens
        return ' '.join(result)               

## Prepare Text Corpus

To train a Language Model, we need a text corpus. Ideally, this corpus should be very large and representative to cover a wide range of n-grams. In this notebook, however, we have to keep it simple. We therefore use publicly available books that can be downloaded from [Project Gutenberg](https://www.gutenberg.org/). Most to all books are available as plain text file for use to use to train a model.

### Download Books from Project Gutenberg

The following code cell downloads a book's plain text file to store it locally in folder `data/`. Feel free to browse the Project Gutenberg website to find the links to you books of choice. The example below is the Sherlock Holmes novel "The Hound of the Baskervilles".

In [4]:
%%time

url = 'https://www.gutenberg.org/files/2852/2852-0.txt' # The Hound of the Baskervilles (Sir Arthur Conan Doyle)

# Download the file using the given URL
r = requests.get(url, allow_redirects=True)

# Specify where to save the file
file_name = 'data/{}'.format(url.split('/')[-1])

# Save the file locally
open(file_name, 'wb').write(r.content)

CPU times: user 44.2 ms, sys: 69 µs, total: 44.2 ms
Wall time: 2.67 s


387870

The output reflect the size of the file in bytes.

### Analyze File using spaCy

The text files for books from Project Gutenberg are not conveniently structures that each line represent a sentence. We therefore need a way to first split the document into sentence. The most convenient here is to simply use spaCy.

Note that this is not a great way to do it as it does not scale to really large corpora. For example, it would be impossible to give a large Wikipedia dump to spaCy. So in practice, a large corpus would first need to be split into reasonably sized chunks. However, spaCy seems to handle at least "The Hound of the Baskervilles"

**Important:** If the code cell below crashes for you, you may want to try a short book first.

In [5]:
%%time 

# Read text file into a string
text = open(file_name).read()

# Set the max. length of a string for spaCy to the length of the document
nlp.max_length = len(text)

# Analyze document (includes splitting the docment into sentences)
doc = nlp(text)

CPU times: user 14.9 s, sys: 701 ms, total: 15.6 s
Wall time: 15.6 s


## Training the Language Model

With the code we have, training a Language Model is easy. We only need to decide on the size of the n-grams. You're of course encouraged to try different n-gram size and see how it affects the generated sentences, but also the number of n-grams.

In [6]:
%%time

#ngram_size = 2
#ngram_size = 3
ngram_size = 4
#ngram_size = 5

model = NgramModel(ngram_size)

for s in doc.sents:
    model.update(s.text)

print("Number of n-grams: {}".format(len(model.ngram_counter)))

Number of n-grams: 66194
CPU times: user 15.1 s, sys: 12.4 ms, total: 15.1 s
Wall time: 15.1 s


## Generating Sentences

After the training, the model is ready to generate new sentences utilizing the probabilities of the n-grams based on the now available n-gram counts. As we don't use smoothing, the only requirement is that the start/seed words represented by `start_context` (more specifically, the last `(self.n - 1)` in `start_context`) are an existing n-gram.

Since we pick the next words (kind of) randomly, executing the same example multiple times will generally yield different sentences. Try different start/seed words and see the generated sentences using multiple runs

In [7]:
# This will not work for "The hound of the Baskervilles" as no sentence in there starts with "I love ..."
#print(model.generate_text(50, start_context=['i', 'love']))

# These should all work for "The hound of the Baskervilles"
print(model.generate_text(50, start_context=['i', 'saw']))
#print(model.generate_text(50, start_context=['he', 'said', 'that']))
#print(model.generate_text(50, start_context=['the', 'train']))
#print(model.generate_text(50, start_context=['sherlock', 'holmes']))
#print(model.generate_text(50, start_context=['the', 'day']))

i saw holmes put his hand to his forehead like a man distracted .


#### Some Observations

If you use "The Hound of the Baskervilles", you should make some basic observations:

* For smaller n-grams (e.g., of size 2 or 3) the sentences are more garbled. This is of course no surprise, give the very small context we use to generate the next word
* Increasing the the size of the n-grams will generally yield more coherent sentences. Again, this is expect as we increase the context for predicting the next word.
* Very quickly, when we increase the size of the n-grams to 5 or larger, the generated sentences are mostly sentence that can be directly found in the book. This is because our corpus is overall very small making most n-grams essentially unique. This also means that the list of candidate word for a context from which we pick the next words is of size 1 most of the time, make the sentence generation more or less deterministic.
* An n-gram size of 4 seems to be the sweet spot here. The generated sentences are rather coherent, and there is still some randomness to yield different sentences when performing multiple runs using the same start/seed words.

Of course, if you use a different corpus, these observations might (slightly) differ.

## Summary

In this notebook we put Language Models to use for text generation. The underlying idea is that the conditional probabilities of the Language model allows us to predict the next word given a current context of words. An obvious next step would be to train a Language Model on a much larger corpus. However, this would require some changes to the code to handle a large corpus as well as significantly increase the training of the model. In general, additional optimization strategies might be applicable. But this is beyond the scope of this notebook.